In [7]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [11]:
#export
from exp.nb_02 import *

def get_data():
    path = Config().data_path()/'mnist'
    with gzip.open(path/'mnist.pkl.gz', 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [12]:
??torch.nn.modules.conv._ConvNd.reset_parameters

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/miniconda3/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [13]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [14]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [15]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [16]:
l1 = nn.Conv2d(1, nh, 5)

In [17]:
x = x_valid[:100]

In [18]:
x.shape

torch.Size([100, 1, 28, 28])

In [19]:
def stats(x): return x.mean(),x.std()

In [20]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [22]:
stats(l1.weight)

(tensor(0.0039, grad_fn=<MeanBackward0>),
 tensor(0.1183, grad_fn=<StdBackward0>))

In [23]:
stats(l1.bias)

(tensor(0.0189, grad_fn=<MeanBackward0>),
 tensor(0.1168, grad_fn=<StdBackward0>))

In [26]:
t = l1(x)

In [27]:
stats(t)

(tensor(0.0288, grad_fn=<MeanBackward0>),
 tensor(0.6973, grad_fn=<StdBackward0>))

In [35]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0169, grad_fn=<MeanBackward0>),
 tensor(1.0981, grad_fn=<StdBackward0>))

In [36]:
import torch.nn.functional as F

In [46]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [68]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5711, grad_fn=<MeanBackward0>),
 tensor(0.9743, grad_fn=<StdBackward0>))

In [69]:
init.kaiming_normal_(l1.weight, a=0.01)
stats(f1(x))

(tensor(0.5320, grad_fn=<MeanBackward0>),
 tensor(0.9705, grad_fn=<StdBackward0>))

In [73]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2239, grad_fn=<MeanBackward0>),
 tensor(0.4371, grad_fn=<StdBackward0>))

In [74]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [75]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [76]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [77]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [78]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [79]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [80]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5728)

In [81]:
1/math.sqrt(3.)

0.5773502691896258

In [82]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [83]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5454, grad_fn=<MeanBackward0>),
 tensor(1.0481, grad_fn=<StdBackward0>))

In [84]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.1875, grad_fn=<MeanBackward0>),
 tensor(0.3212, grad_fn=<StdBackward0>))

In [104]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [120]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [121]:
y = y_valid[:100].float()

In [122]:
t = m(x)
stats(t)

(tensor(-0.0522, grad_fn=<MeanBackward0>),
 tensor(0.0095, grad_fn=<StdBackward0>))

In [123]:
l = mse(t,y)
l.backward()

In [124]:
stats(m[0].weight.grad)

(tensor(0.0061), tensor(0.0382))

In [125]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \sqrt{\frac{6}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
      

In [132]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [133]:
t = m(x)
stats(t)

(tensor(0.1489, grad_fn=<MeanBackward0>),
 tensor(0.2053, grad_fn=<StdBackward0>))

In [134]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(-0.8188), tensor(0.9915))

## Export

In [ ]:
!./notebook2script.py 02a_why_sqrt5.ipynb